In [102]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import datetime
import time
import re

There are three parts to this webscraper.

1. Webscrape Moves with power, type, accuracy PP and description
2. Moves With Their Action Speed
3. Merge Two datasets for a comprehensive Pokemon Arceus moves list

## 1) Moves List With Power, Type, Accuracy, PP & Description

In [78]:
# Moves with power, accuracy and description Link:

url_moves = 'https://www.serebii.net/legendsarceus/attacks.shtml'

In [79]:
# Open up arceus.gg website:

driver = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe')
driver.get(url_moves)

# Maximize Window:
driver.maximize_window()
driver.implicitly_wait(5)

In [80]:
# Go to moves table:

#moves_table = driver.find_element_by_xpath("//*[@id='wptb-table-id-81812']/table")

#moves_table.location_once_scrolled_into_view

In [81]:
# Move Names:

moves_one = [x.text for x in 
             driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr[position() > 1]/td[1]/a')]

In [158]:
# Type, go to link in alt and use regex to extract

types_one = [x.get_attribute('src') for x in 
             driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr[position() > 1]/td[2]/img')]

In [97]:
# Class (Physical or Special), go to link in altand use regex to extract

classes_one = [x.get_attribute('src') for x in 
               driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr/td[3]/img')]

In [91]:
# PP

pp_one = [x.text for x in 
          driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr[position() > 1]/td[4]')]

In [85]:
# Base Power

powers_one = [x.text for x in 
             driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr[position() > 1]/td[5]')]

In [86]:
# Accuracy:

acc_one = [x.text for x in 
           driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr[position() > 1]/td[6]')]

In [87]:
# Move Descriptions:

desc_one = [x.text for x in 
            driver.find_elements_by_xpath('/html/body/div/div[2]/main/table/tbody/tr[position() > 1]/td[7]')]

### Extract Type from Link, Extract Class from Link

In [163]:
# Remove https://www.serebii.net/pokedex-bw/type/ part and the .gif part, extract type only:

types_list_one = [re.sub('^.+\/|.gif$', "", x).title()  for x in types_one]

In [165]:
types_list_one[1:10]

['Poison',
 'Poison',
 'Flying',
 'Flying',
 'Flying',
 'Rock',
 'Water',
 'Water',
 'Ghost']

In [170]:
# Remove https://www.serebii.net/pokedex-bw/type/ part and the .gif part, extract type only:

classes_list_one = [re.sub('^.+\/|.png$', "", x).title()  for x in classes_one]

In [171]:
classes_list_one[1:10]

['Other',
 'Special',
 'Physical',
 'Special',
 'Special',
 'Special',
 'Physical',
 'Physical',
 'Physical']

## Change Dash To Base Power 0 For Status Moves

In [179]:
powers_list_one = [re.sub('--', "0", x) for x in powers_one]

### Make Dataframe 1

In [182]:
## Make dataframe & close driver one

pokemon_legends_moves_df = pd.DataFrame({
    'Move': moves_one,
    'Type': types_list_one,
    'Class': classes_list_one,
    'PP': pp_one ,
    'Power': powers_list_one,
    'Accuracy': acc_one ,
    'Description': desc_one
})

driver.close()

In [183]:
pokemon_legends_moves_df.head(15)

,Move,Type,Class,PP,Power,Accuracy,Description
0,Absorb,Grass,Special,20,30,100,The user attacks by draining the target's nutr...
1,Acid Armor,Poison,Other,20,0,101,The user alters its cellular structure to liqu...
2,Acid Spray,Poison,Special,20,40,100,The user spits fluid that works to melt the ta...
3,Aerial Ace,Flying,Physical,20,60,101,"The user confounds the target with speed, then..."
4,Air Cutter,Flying,Special,15,60,95,The user launches razor-like wind to slash the...
5,Air Slash,Flying,Special,10,75,95,The user attacks with a blade of air that slic...
6,Ancient Power,Rock,Special,15,60,100,The user attacks with a primeval power. This m...
7,Aqua Jet,Water,Physical,20,40,100,"The user lunges at the target to deal damage, ..."
8,Aqua Tail,Water,Physical,10,85,90,The user attacks by swinging its tail as if it...
9,Astonish,Ghost,Physical,25,30,100,The user attacks the target while crying out i...


In [195]:
# Convert to numeric columns for PP, Power & Accuracy:

cols_one = ['PP', "Power", 'Accuracy']

pokemon_legends_moves_df[cols_one] = pd.to_numeric(pokemon_legends_moves_df[cols_one].stack(), 
                                                   errors='coerce').unstack()

In [197]:
# Check data types
pokemon_legends_moves_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Move         177 non-null    object
 1   Type         177 non-null    object
 2   Class        177 non-null    object
 3   PP           177 non-null    int64 
 4   Power        177 non-null    int64 
 5   Accuracy     177 non-null    int64 
 6   Description  177 non-null    object
dtypes: int64(3), object(4)
memory usage: 9.8+ KB


## 2) Action Speed Information & Moves In Pokemon Legends Arceus

Battle mechanics in Pokemon Legends Arceus is a bit different from the previous Pokemon games. 

There is a wait time for Pokemon users and target Pokemon. It is a bit confusing but the details are from this site:

https://bulbapedia.bulbagarden.net/wiki/Action_speed

A higher action time is bad as it takes longer for you to get another turn. Using strong style makes it longer to get to another turn. Agile style does have weaker damage but you can possibly get another turn before the opponent. 

In [186]:
# New URL and open new driver

url_action_speed = 'https://bulbapedia.bulbagarden.net/wiki/Action_speed'

# Open up arceus.gg website:

driver = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe')
driver.get(url_action_speed)

# Maximize Window:
driver.maximize_window()
driver.implicitly_wait(5)

In [190]:
# Move Name:
moves_two = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[1]/a')]

# Move Type:
types_list_two = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[2]/a')]

# Classes/Category list:
classes_list_two = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[3]/a')]

In [188]:
## User's Action Time:

# Regular
user_reg_action_time = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[4]')]

# Agile
user_agile_action_time = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[5]')]

# Strong
user_strong_action_time = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[6]')]

In [189]:
## Target Pokemon Action Time

# Reg/Strong
tgt_nonagile_action_time = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[7]')]

# Agile Style
tgt_agile_action_time = [x.text for x in driver.find_elements_by_xpath('//*/table[3]/tbody/tr/td[8]')]

driver.implicit_wait(5)
driver.close()

In [201]:
## Make dataframe & close driver two

action_times_df = pd.DataFrame({
                    'Move': moves_two,
                    'Type': types_list_two,
                    'Class': classes_list_two,
                    'User Reg Action Time': user_reg_action_time,
                    'User Agile Action Time': user_agile_action_time,
                    'User Strong Action Time': user_strong_action_time,
                    'Target NonAgile Action Time': tgt_nonagile_action_time,
                    'Target Agile Action Time': tgt_agile_action_time
})


In [202]:
action_times_df.head(10)

,Move,Type,Class,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
0,Fire Punch,Fire,Physical,0,0,5,0,3
1,Ice Punch,Ice,Physical,0,0,5,0,3
2,Thunder Punch,Electric,Physical,0,0,5,0,3
3,Swords Dance,Normal,Status,-3,-4,5,0,0
4,Gust,Flying,Special,0,0,5,0,3
5,Tackle,Normal,Physical,0,0,5,0,3
6,Double-Edge,Normal,Physical,0,0,5,0,3
7,Poison Sting,Poison,Physical,0,0,5,0,3
8,Pin Missile,Bug,Physical,0,0,5,0,3
9,Bite,Dark,Physical,0,0,5,3,5


In [203]:
# Convert to numeric columns for Action Times:

cols_two = ['User Reg Action Time', "User Agile Action Time", 'User Strong Action Time',
            'Target NonAgile Action Time', 'Target Agile Action Time']

action_times_df[cols_two] = pd.to_numeric(action_times_df[cols_two].stack(), 
                                          errors='coerce').unstack()

In [204]:
action_times_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Move                         177 non-null    object
 1   Type                         177 non-null    object
 2   Class                        177 non-null    object
 3   User Reg Action Time         177 non-null    int64 
 4   User Agile Action Time       177 non-null    int64 
 5   User Strong Action Time      177 non-null    int64 
 6   Target NonAgile Action Time  177 non-null    int64 
 7   Target Agile Action Time     177 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 11.2+ KB


In [209]:
# Sort by Move ABC order:

action_times_df = action_times_df.sort_values('Move', ascending = True)

In [210]:
action_times_df

,Move,Type,Class,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
16,Absorb,Grass,Special,0,0,5,0,3
42,Acid Armor,Poison,Status,-3,-4,5,0,0
134,Acid Spray,Poison,Special,0,0,5,0,3
74,Aerial Ace,Flying,Physical,0,0,5,0,3
70,Air Cutter,Flying,Special,0,0,5,0,3
...,...,...,...,...,...,...,...,...
138,Wild Charge,Electric,Physical,0,0,5,0,3
173,Wildbolt Storm,Electric,Special,0,0,5,0,3
121,Wood Hammer,Grass,Physical,0,0,5,0,3
92,X-Scissor,Bug,Physical,0,0,5,0,3


In [217]:
action_times_df.loc[:, ~action_times_df.columns.isin(['Type', 'Class'])]

,Move,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
16,Absorb,0,0,5,0,3
42,Acid Armor,-3,-4,5,0,0
134,Acid Spray,0,0,5,0,3
74,Aerial Ace,0,0,5,0,3
70,Air Cutter,0,0,5,0,3
...,...,...,...,...,...,...
138,Wild Charge,0,0,5,0,3
173,Wildbolt Storm,0,0,5,0,3
121,Wood Hammer,0,0,5,0,3
92,X-Scissor,0,0,5,0,3


## 3) Merge/Combine Two Datasets Of Moves

In [223]:
# Merge data frames, left df is Move, Type, Class, PP, Power Accuracy and right df are action times

full_moves_df = pd.merge(pokemon_legends_moves_df, 
                         action_times_df.loc[:, ~action_times_df.columns.isin(['Type', 'Class'])], 
                         on = "Move", 
                         how = "left")

In [224]:
full_moves_df

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
0,Absorb,Grass,Special,20,30,100,The user attacks by draining the target's nutr...,0,0,5,0,3
1,Acid Armor,Poison,Other,20,0,101,The user alters its cellular structure to liqu...,-3,-4,5,0,0
2,Acid Spray,Poison,Special,20,40,100,The user spits fluid that works to melt the ta...,0,0,5,0,3
3,Aerial Ace,Flying,Physical,20,60,101,"The user confounds the target with speed, then...",0,0,5,0,3
4,Air Cutter,Flying,Special,15,60,95,The user launches razor-like wind to slash the...,0,0,5,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
172,Wild Charge,Electric,Physical,10,85,100,The user shrouds itself in electricity and sma...,0,0,5,0,3
173,Wildbolt Storm,Electric,Special,5,95,80,The user summons a thunderous tempest and sava...,0,0,5,0,3
174,Wood Hammer,Grass,Physical,5,100,100,The user slams its rugged body into the target...,0,0,5,0,3
175,X-Scissor,Bug,Physical,10,80,100,The user slashes at the target by crossing its...,0,0,5,0,3


In [225]:
full_moves_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177 entries, 0 to 176
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Move                         177 non-null    object
 1   Type                         177 non-null    object
 2   Class                        177 non-null    object
 3   PP                           177 non-null    int64 
 4   Power                        177 non-null    int64 
 5   Accuracy                     177 non-null    int64 
 6   Description                  177 non-null    object
 7   User Reg Action Time         177 non-null    int64 
 8   User Agile Action Time       177 non-null    int64 
 9   User Strong Action Time      177 non-null    int64 
 10  Target NonAgile Action Time  177 non-null    int64 
 11  Target Agile Action Time     177 non-null    int64 
dtypes: int64(8), object(4)
memory usage: 18.0+ KB


### Save to .csv File:

In [227]:
full_moves_df.to_csv("Pokemon_Legends_Arceus_Moves_Stats.csv", index=False)

## Optional: Data Analysis & Filtering

In [233]:
import pandas as pd

# Load data:

df = pd.read_csv("Pokemon_Legends_Arceus_Moves_Stats.csv")

In [234]:
df.head(10)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
0,Absorb,Grass,Special,20,30,100,The user attacks by draining the target's nutr...,0,0,5,0,3
1,Acid Armor,Poison,Other,20,0,101,The user alters its cellular structure to liqu...,-3,-4,5,0,0
2,Acid Spray,Poison,Special,20,40,100,The user spits fluid that works to melt the ta...,0,0,5,0,3
3,Aerial Ace,Flying,Physical,20,60,101,"The user confounds the target with speed, then...",0,0,5,0,3
4,Air Cutter,Flying,Special,15,60,95,The user launches razor-like wind to slash the...,0,0,5,0,3
5,Air Slash,Flying,Special,10,75,95,The user attacks with a blade of air that slic...,0,0,5,3,5
6,Ancient Power,Rock,Special,15,60,100,The user attacks with a primeval power. This m...,0,0,5,0,3
7,Aqua Jet,Water,Physical,20,40,100,"The user lunges at the target to deal damage, ...",-4,-4,2,0,3
8,Aqua Tail,Water,Physical,10,85,90,The user attacks by swinging its tail as if it...,0,0,5,0,3
9,Astonish,Ghost,Physical,25,30,100,The user attacks the target while crying out i...,0,0,5,3,5


### Powerful Moves

In [254]:
df.sort_values('Power', ascending = False).head(20)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
128,Self-Destruct,Normal,Physical,5,150,100,The user causes an explosion to inflict damage...,0,0,5,0,3
169,Volt Tackle,Electric,Physical,5,120,100,The user electrifies itself and charges the ta...,0,0,5,0,3
58,Giga Impact,Normal,Physical,5,120,90,The user charges at the target using every bit...,5,2,8,0,3
60,Head Smash,Rock,Physical,5,120,80,"The user attacks the target with a hazardous, ...",0,0,5,0,3
26,Chloroblast,Grass,Special,5,120,95,The user launches its amassed chlorophyll to i...,5,2,8,0,3
67,Hyper Beam,Normal,Special,5,120,90,The target is attacked with a powerful beam.,5,2,8,0,3
147,Steel Beam,Steel,Special,5,120,95,The user fires a beam of steel that it collect...,0,0,5,0,3
121,Roar of Time,Dragon,Special,5,120,90,The user blasts the target with power that dis...,5,2,8,0,3
103,Overheat,Fire,Special,5,110,90,The user attacks the target at full power. The...,0,0,5,0,3
82,Leaf Storm,Grass,Special,5,110,90,The user whips up a storm of leaves around the...,0,0,5,0,3


## Top 10 Physical Moves

In [258]:
df[df['Class'] == 'Physical'].sort_values('Power', ascending = False).head(10)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
128,Self-Destruct,Normal,Physical,5,150,100,The user causes an explosion to inflict damage...,0,0,5,0,3
58,Giga Impact,Normal,Physical,5,120,90,The user charges at the target using every bit...,5,2,8,0,3
60,Head Smash,Rock,Physical,5,120,80,"The user attacks the target with a hazardous, ...",0,0,5,0,3
169,Volt Tackle,Electric,Physical,5,120,100,The user electrifies itself and charges the ta...,0,0,5,0,3
131,Shadow Force,Ghost,Physical,5,100,100,The user cloaks itself in darkness as it attac...,3,1,5,0,3
37,Double-Edge,Normal,Physical,5,100,100,"A reckless, life-risking tackle in which the u...",0,0,5,0,3
61,Headlong Rush,Ground,Physical,5,100,100,The user smashes into the target in a full-bod...,0,0,5,0,3
31,Crush Grip,Normal,Physical,5,100,100,The target is crushed with great force. The mo...,0,0,5,3,5
27,Close Combat,Fighting,Physical,5,100,100,"The user fights the target up close, inflictin...",0,0,5,0,3
149,Stone Edge,Rock,Physical,5,100,80,The user stabs the target with sharpened stone...,0,0,5,0,3


## Top 10 Special Moves

In [257]:
df[df['Class'] == 'Special'].sort_values('Power', ascending = False).head(10)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
121,Roar of Time,Dragon,Special,5,120,90,The user blasts the target with power that dis...,5,2,8,0,3
67,Hyper Beam,Normal,Special,5,120,90,The target is attacked with a powerful beam.,5,2,8,0,3
26,Chloroblast,Grass,Special,5,120,95,The user launches its amassed chlorophyll to i...,5,2,8,0,3
147,Steel Beam,Steel,Special,5,120,95,The user fires a beam of steel that it collect...,0,0,5,0,3
38,Draco Meteor,Dragon,Special,5,110,90,Comets are summoned down from the sky onto the...,0,0,5,0,3
103,Overheat,Fire,Special,5,110,90,The user attacks the target at full power. The...,0,0,5,0,3
82,Leaf Storm,Grass,Special,5,110,90,The user whips up a storm of leaves around the...,0,0,5,0,3
50,Fire Blast,Fire,Special,5,100,85,The target is attacked with an intense blast o...,0,0,5,0,3
127,Seed Flare,Grass,Special,5,100,85,The user emits a shock wave from its body to a...,0,0,5,0,3
80,Judgment,Normal,Special,5,100,100,The user releases countless shots of light at ...,0,0,5,0,3


## Attacks That Don't Miss

In [263]:
df[(df['Accuracy'] == 101) & (df['Class'] != 'Other')].sort_values('Power', ascending = False)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
10,Aura Sphere,Fighting,Special,10,80,101,The user lets loose a blast of aura power from...,0,0,5,0,3
32,Dark Pulse,Dark,Special,10,80,101,The user releases a horrible aura imbued with ...,0,0,5,0,3
40,Dragon Pulse,Dragon,Special,10,80,101,The target is attacked with a shock wave gener...,0,0,5,0,3
3,Aerial Ace,Flying,Physical,20,60,101,"The user confounds the target with speed, then...",0,0,5,0,3
88,Magical Leaf,Grass,Special,20,60,101,The user scatters curious leaves that chase th...,0,0,5,0,3
153,Swift,Normal,Special,20,60,101,Star-shaped rays are shot at the target. This ...,0,0,5,0,3
170,Water Pulse,Water,Special,20,60,101,The user attacks the target with a pulsing bla...,0,0,5,0,3
150,Struggle,Normal,Physical,1,50,101,This attack is used in desperation only if the...,0,0,0,0,0


### Status Moves

In [236]:
# Status moves where user action time is reduced. Lower action time is less wait for another turn.

df[df['Class'] == 'Other'].sort_values('User Reg Action Time')

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
1,Acid Armor,Poison,Other,20,0,101,The user alters its cellular structure to liqu...,-3,-4,5,0,0
154,Swords Dance,Normal,Other,20,0,101,A frenetic dance to uplift the fighting spirit...,-3,-4,5,0,0
98,Nasty Plot,Dark,Other,20,0,101,The user stimulates its brain by thinking bad ...,-3,-4,5,0,0
77,Iron Defense,Steel,Other,20,0,101,"The user hardens its body's surface like iron,...",-3,-4,5,0,0
57,Focus Energy,Normal,Other,20,0,101,The user takes a deep breath and focuses so th...,-3,-4,5,0,0
113,Power Shift,Normal,Other,10,0,101,The user swaps its offensive and defensive stats.,-3,-4,5,0,0
11,Baby-Doll Eyes,Fairy,Other,20,0,100,The user stares at the target with its baby-do...,-3,-4,5,0,3
36,Double Hit,Normal,Other,10,0,101,The user adopts a hard-hitting stance to incre...,-3,-4,5,0,0
168,Victory Dance,Fighting,Other,10,0,101,The user performs a dance to usher in victory....,-2,-3,5,0,0
23,Calm Mind,Psychic,Other,10,0,101,The user quietly focuses its mind and calms it...,-2,-3,5,0,0


In [238]:
# Status moves where target action time is higher under agile style. 
# Higher action time for opponent means target waits longer for a turn

df[df['Class'] == 'Other'].sort_values('Target Agile Action Time', ascending = False)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
152,Stun Spore,Grass,Other,20,0,80,The user scatters a cloud of numbing powder th...,0,0,5,0,3
162,Thunder Wave,Electric,Other,20,0,90,The user launches a weak jolt of electricity t...,0,0,5,0,3
136,Sleep Powder,Grass,Other,20,0,80,The user scatters a cloud of sleep-inducing po...,0,0,5,0,3
33,Dark Void,Dark,Other,10,0,90,The target is dragged into a world of total da...,5,2,8,0,3
11,Baby-Doll Eyes,Fairy,Other,20,0,100,The user stares at the target with its baby-do...,-3,-4,5,0,3
109,Poison Powder,Poison,Other,20,0,80,The user scatters a cloud of toxic powder that...,0,0,5,0,3
68,Hypnosis,Psychic,Other,20,0,70,The user employs hypnotic suggestion to make t...,0,0,5,0,3
107,Poison Gas,Poison,Other,20,0,90,The user sprays a cloud of toxic gas that pois...,0,0,5,0,3
144,Spore,Grass,Other,10,0,100,The user scatters a burst of spores that makes...,0,0,5,0,3
139,Soft-Boiled,Normal,Other,10,0,101,The user restores its own HP.,0,-3,5,0,0


Note that Baby-Doll Eyes moves decreases action time for user and increases action time on target.

### Attacks That Reduce User Action Speed

Moves that reduce user action speed.

In [248]:
df[(df['Class'] != 'Other') & (df['User Reg Action Time'] < 0)]

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
7,Aqua Jet,Water,Physical,20,40,100,"The user lunges at the target to deal damage, ...",-4,-4,2,0,3
22,Bullet Punch,Steel,Physical,20,40,100,The user strikes the target with tough punches...,-4,-4,2,0,3
46,Esper Wing,Psychic,Special,10,75,90,The user slashes the target with aura-enriched...,-4,-4,2,0,3
73,Ice Shard,Ice,Physical,20,40,100,The user flash-freezes chunks of ice and hurls...,-4,-4,2,0,3
87,Mach Punch,Fighting,Physical,20,40,100,The user throws a punch at blinding speed. Thi...,-4,-4,2,0,3
117,Quick Attack,Normal,Physical,20,40,100,"The user lunges at the target to deal damage, ...",-4,-4,2,0,3
132,Shadow Sneak,Ghost,Physical,20,40,100,The user extends its shadow and attacks the ta...,-4,-4,2,0,3
171,Wave Crash,Water,Physical,10,75,100,The user shrouds itself in water and slams int...,-4,-4,2,0,3


### Slower Target Moves (Formerly Flinch)

Target would have higher action speed instead of flinch. Agile style would make it from 3 to 5 for target Pokemon.

In [252]:
df[(df['Class'] != 'Other') & (df['Target NonAgile Action Time'] > 0)].sort_values('Power', ascending = False)

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
93,Mountain Gale,Ice,Physical,5,100,85,The user hurls giant chunks of ice at the targ...,5,2,8,3,5
31,Crush Grip,Normal,Physical,5,100,100,The target is crushed with great force. The mo...,0,0,5,3,5
176,Zen Headbutt,Psychic,Physical,10,80,90,The user focuses its willpower to its head and...,0,0,5,3,5
74,Icicle Crash,Ice,Physical,10,80,90,The user attacks by crashing large icicles ont...,0,0,5,3,5
78,Iron Head,Steel,Physical,10,80,100,The user slams the target with its steel-hard ...,0,0,5,3,5
122,Rock Slide,Rock,Physical,10,75,90,Large boulders are hurled at the target to inf...,0,0,5,3,5
5,Air Slash,Flying,Special,10,75,95,The user attacks with a blade of air that slic...,0,0,5,3,5
47,Extrasensory,Psychic,Special,15,70,100,"The user attacks with an odd, unseeable power.",0,0,5,3,5
51,Fire Fang,Fire,Physical,15,65,95,The user bites the target with flame-cloaked f...,0,0,5,3,5
159,Thunder Fang,Electric,Physical,15,65,95,The user bites the target with electrified fan...,0,0,5,3,5


### Slower User Action Speed Moves (Formerly Recharge)

In [250]:
df[(df['Class'] != 'Other') & (df['User Reg Action Time'] > 0)]

,Move,Type,Class,PP,Power,Accuracy,Description,User Reg Action Time,User Agile Action Time,User Strong Action Time,Target NonAgile Action Time,Target Agile Action Time
26,Chloroblast,Grass,Special,5,120,95,The user launches its amassed chlorophyll to i...,5,2,8,0,3
58,Giga Impact,Normal,Physical,5,120,90,The user charges at the target using every bit...,5,2,8,0,3
67,Hyper Beam,Normal,Special,5,120,90,The target is attacked with a powerful beam.,5,2,8,0,3
93,Mountain Gale,Ice,Physical,5,100,85,The user hurls giant chunks of ice at the targ...,5,2,8,3,5
121,Roar of Time,Dragon,Special,5,120,90,The user blasts the target with power that dis...,5,2,8,0,3
131,Shadow Force,Ghost,Physical,5,100,100,The user cloaks itself in darkness as it attac...,3,1,5,0,3
